In [4]:
import json
from pathlib import Path
from copy import deepcopy
import cv2
from tqdm import tqdm
import os

# =============== 用户配置 ===============
DATA_ROOT = Path("/home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/00_test/00_try/del")          # 数据集根目录（改成你的）
IMG_DIR = DATA_ROOT / "data"       # 原图目录
COCO_JSON = DATA_ROOT / "labels.json" # 原始 COCO 标注文件

OUT_IMG_DIR = DATA_ROOT / "images_640_ov20"   # 输出小图目录
OUT_JSON = DATA_ROOT / "coco_crops_ov20.json" # 输出新的 COCO 标注

CROP_SIZE = 640          # 小图尺寸：640 x 640
OVERLAP_RATIO = 0.2      # overlap 20%
STRIDE = int(CROP_SIZE * (1 - OVERLAP_RATIO))  # 实际步长 = 512
KEEP_RATIO = 0.9         # 裁剪后保留面积比例 < 这个值就丢掉该框
# =======================================


def crop_bbox_to_tile(bbox, tile_x, tile_y, tile_size):
    x, y, w, h = bbox
    x2, y2 = x + w, y + h

    tx1, ty1 = tile_x, tile_y
    tx2, ty2 = tile_x + tile_size, ty1 + tile_size

    ix1 = max(x, tx1)
    iy1 = max(y, ty1)
    ix2 = min(x2, tx2)
    iy2 = min(y2, ty2)

    inter_w = max(0.0, ix2 - ix1)
    inter_h = max(0.0, iy2 - iy1)

    if inter_w <= 0 or inter_h <= 0:
        return None, 0.0

    inter_area = inter_w * inter_h
    orig_area = max(w * h, 1e-6)
    keep_ratio = inter_area / orig_area

    return [ix1 - tx1, iy1 - ty1, inter_w, inter_h], keep_ratio


def make_positions(length, crop_size, stride):
    if length <= crop_size:
        return [0]
    pos_list, pos = [], 0
    while pos + crop_size < length:
        pos_list.append(pos)
        pos += stride

    last = length - crop_size
    if not pos_list or pos_list[-1] != last:
        pos_list.append(last)

    return pos_list


def main():
    OUT_IMG_DIR.mkdir(parents=True, exist_ok=True)

    with COCO_JSON.open("r", encoding="utf-8") as f:
        coco = json.load(f)

    images = coco["images"]
    annotations = coco["annotations"]
    categories = coco["categories"]

    anns_by_img = {}
    for ann in annotations:
        anns_by_img.setdefault(ann["image_id"], []).append(ann)

    new_images = []
    new_annotations = []
    new_image_id = 1
    new_ann_id = 1

    for img_info in tqdm(images, desc="Processing images"):
        img_id = img_info["id"]
        file_name = img_info["file_name"]
        img_path = IMG_DIR / file_name

        img = cv2.imread(str(img_path))
        if img is None:
            print(f"\n[WARN] 无法读取图片: {img_path}")
            continue

        h, w = img.shape[:2]
        xs = make_positions(w, CROP_SIZE, STRIDE)
        ys = make_positions(h, CROP_SIZE, STRIDE)

        orig_anns = anns_by_img.get(img_id, [])

        for ty in ys:
            for tx in xs:
                tile = img[ty:ty + CROP_SIZE, tx:tx + CROP_SIZE]

                new_file = f"{Path(file_name).stem}_{tx}_{ty}{Path(file_name).suffix}"
                out_path = OUT_IMG_DIR / new_file

                # 临时先保存图片（如果为空等会删）
                cv2.imwrite(str(out_path), tile)

                # 收集该 tile 的标注
                tile_ann_list = []

                for ann in orig_anns:
                    new_bbox, ratio = crop_bbox_to_tile(ann["bbox"], tx, ty, CROP_SIZE)
                    if new_bbox is None or ratio < KEEP_RATIO:
                        continue

                    new_ann = deepcopy(ann)
                    new_ann["id"] = new_ann_id
                    new_ann["image_id"] = new_image_id
                    new_ann["bbox"] = new_bbox
                    new_ann["area"] = new_bbox[2] * new_bbox[3]

                    tile_ann_list.append(new_ann)
                    new_ann_id += 1

                # ⭐⭐⭐ 去除空 tile：如果没有标注，直接删除 tile 图片 ⭐⭐⭐
                if len(tile_ann_list) == 0:
                    if out_path.exists():
                        os.remove(out_path)
                    continue  # 不写入 new_images / new_annotations

                # 写入 new_images 记录
                new_images.append({
                    "id": new_image_id,
                    "file_name": new_file,
                    "width": CROP_SIZE,
                    "height": CROP_SIZE
                })

                # 写入标注
                new_annotations.extend(tile_ann_list)

                new_image_id += 1

    # 保存新的 COCO
    new_coco = {
        "images": new_images,
        "annotations": new_annotations,
        "categories": categories
    }

    with OUT_JSON.open("w", encoding="utf-8") as f:
        json.dump(new_coco, f, ensure_ascii=False, indent=2)

    print("\n==== 完成 20% overlap 切图（自动去除空 tile） ====")
    print(f"有效 tiles 数: {len(new_images)}")
    print(f"标注数量:     {len(new_annotations)}")
    print(f"输出目录:     {OUT_IMG_DIR}")
    print(f"输出 COCO：   {OUT_JSON}")


if __name__ == "__main__":
    main()



Processing images: 100%|██████████| 323/323 [01:30<00:00,  3.58it/s]


==== 完成 20% overlap 切图（自动去除空 tile） ====
有效 tiles 数: 759
标注数量:     781
输出目录:     /home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/00_test/00_try/del/images_640_ov20
输出 COCO：   /home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/00_test/00_try/del/coco_crops_ov20.json
